In [ ]:
!pip install transformers peft bitsandbytes accelerate trl pydantic-settings scipy codebleu

In [ ]:
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps trl peft accelerate bitsandbytes
!pip install -q xformers==0.0.22.post7

In [1]:
# DEFINE QUANTIZATION HERE. Choose from ("none" | "8bit" | "4bit")
QUANTIZATION = "4bit"#DO NOT CHANGE
DATASET = "codeparrot/xlcost-text-to-code" #DO NOT CHANGE
TESTSIZE = 200

In [2]:
#CHANGE THE FOLLOWING
EXPERT = "python" #python, cpp, java, javascript, etc.
MODELTYPE = "FINETUNED" #BASE or FINETUNED or MOE
model_id = "mistralai/Mistral-7B-v0.1"
loramodel = f"mistral-7b-{EXPERT}-LORA-4bit"
lorarepo = f'santoshsto/{loramodel}'
newmodel = f'mistral-7b-{EXPERT}-{MODELTYPE}-4bit'
PARTITION = "Python-program-level" #java-program-level etc.

In [3]:
from huggingface_hub import login
login(token="")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [4]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [5]:
from datasets import load_dataset
data = load_dataset(DATASET, PARTITION)

Generating validation split: 100%|██████████| 472/472 [00:00<00:00, 213055.48 examples/s]


In [6]:
next(iter(data["test"]))

{'text': 'Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array',
 'code': "def minSum ( A , N ) : NEW_LINE INDENT mp = { } NEW_LINE sum = 0 NEW_LINE for i in range ( N ) : NEW_LINE INDENT sum += A [ i ] NEW_LINE if A [ i ] in mp : NEW_LINE INDENT mp [ A [ i ] ] += 1 NEW_LINE DEDENT else : NEW_LINE INDENT mp [ A [ i ] ] = 1 NEW_LINE DEDENT DEDENT minSum = float ( ' inf ' ) NEW_LINE for it in mp : NEW_LINE INDENT minSum = min ( minSum , sum - ( it * mp [ it ] ) ) NEW_LINE DEDENT return minSum NEW_LINE DEDENT arr = [ 4 , 5 , 6 , 6 ] NEW_LINE N = len ( arr ) NEW_LINE print ( minSum ( arr , N ) ) NEW_LINE"}

In [7]:
train_dataset = data["train"]
eval_dataset = data["validation"]
test_dataset = data["test"]

In [8]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text', 'code'],
    num_rows: 9263
})
Dataset({
    features: ['text', 'code'],
    num_rows: 472
})
Dataset({
    features: ['text', 'code'],
    num_rows: 887
})


## Model Merge

In [9]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [01:49<00:00, 54.92s/it]


In [ ]:
from torch.nn.utils import parameters_to_vector

total_params = parameters_to_vector(base_model.parameters()).numel()
total_params

In [10]:
print(f'Base Model: {model_id}')
print(f'Adapeter: {lorarepo}')
model_to_merge = PeftModel.from_pretrained(base_model, lorarepo)
model = model_to_merge.merge_and_unload()

Base Model: mistralai/Mistral-7B-v0.1
Adapeter: santoshsto/mistral-7b-python-LORA-4bit


In [ ]:
model.push_to_hub(newmodel, use_temp_dir=False)
tokenizer.push_to_hub(newmodel, use_temp_dir=False)

## Evaluation

In [ ]:
eval_prompt = f"""<s>[INST] Write a {EXPERT} program to complete the following. Minimum sum possible by removing all occurrences of any array element | Function to find minimum sum after deletion ; Stores frequency of array elements ; Traverse the array ; Calculate sum ; Update frequency of the current element ; Stores the minimum sum required ; Traverse map ; Find the minimum sum obtained ; Return minimum sum ; Input array ; Size of array [/INST] \\n code:</s>"""

In [ ]:
model.eval()
with torch.no_grad():
  input_ids = tokenizer(eval_prompt, return_tensors="pt", truncation=True).input_ids.cuda()
  outputs = model.generate(input_ids=input_ids, max_new_tokens=256, do_sample=True, top_p=0.9,temperature=0.5)

  print(f"Prompt:\n{eval_prompt}\n")
  print(f"\nGenerated response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(eval_prompt):]}")
  print(f'''\nGround truth:{next(iter(test_dataset))}''')